In [55]:
from ortools.sat.python import cp_model
from pydantic import BaseModel, PositiveInt, NonNegativeFloat, model_validator
from math import floor, ceil

In [56]:
class KnapsackInstance(BaseModel):
    weights: list[PositiveInt]  # the weight of each item
    values: list[PositiveInt]  # the value of each item
    capacity: PositiveInt  # the capacity of the knapsack

    @model_validator(mode="after")
    def check_lengths(cls, v):
        if len(v.weights) != len(v.values):
            raise ValueError("Mismatch in number of weights and values.")
        return v


class KnapsackSolverConfig(BaseModel):
    time_limit: PositiveInt = 900  # Solver time limit in seconds
    opt_tol: NonNegativeFloat = 0.01  # Optimality tolerance (1% gap allowed)
    log_search_progress: bool = False  # Whether to log search progress


class KnapsackSolution(BaseModel):
    selected_items: list[int]  # Indices of the selected items
    objective: float  # Objective value of the solution
    upper_bound: float  # Upper bound of the solution

In [57]:
class MultiObjectiveKnapsackSolver:
    def __init__(self, instance: KnapsackInstance, config: KnapsackSolverConfig):
        self.instance = instance
        self.config = config
        self.model = cp_model.CpModel()
        self.n = len(instance.weights)
        self.x = [self.model.new_bool_var(f"x_{i}") for i in range(self.n)]
        self._objective = 0
        self._build_model()
        self.solver = cp_model.CpSolver()

    # ----------------------------------------
    # Objective functions
    # ----------------------------------------

    def set_maximize_value_objective(self):
        self._objective = sum(
            value * x_i for value, x_i in zip(self.instance.values, self.x)
        )
        self.model.maximize(self._objective)

    def set_minimize_weight_objective(self):
        self._objective = sum(
            weight * x_i for weight, x_i in zip(self.instance.weights, self.x)
        )
        self.model.minimize(self._objective)

    def _set_solution_as_hint(self):
        """
        Set the current solution as a hint for the next solve.
        Important: If you add constraints that are not satisfied by the current solution,
        this is not always useful. It is primarily useful when you want to use this solution
        as a starting point for another solve with a different objective function.
        """
        for i, v in enumerate(self.model.proto.variables):
            v_ = self.model.get_int_var_from_proto_index(i)
            # We are using the internal protobuf here, so future versions of OR-Tools
            # might behave differently. Adding an assertion to verify (deactivate in production for speed).
            assert v.name == v_.name, "Should be the same variable"
            self.model.add_hint(v_, self.solver.value(v_))

    def fix_current_objective(self, ratio: float = 1.0):
        """
        Fix the current objective value to not degenerate.
        You can then exchange the objective function with another one,
        while the value of the previous one is fixed.

        These constraints will not violate the current solution, making hints
        extremely useful.
        """
        if ratio == 1.0:
            self.model.add(self._objective == self.solver.objective_value)
        elif ratio > 1.0:
            self.model.add(self._objective <= ceil(self.solver.objective_value * ratio))
        else:
            # If the ratio is less than 1, we have a maximization problem, thus, we
            # need to lower bound.
            self.model.add(
                self._objective >= floor(self.solver.objective_value * ratio)
            )

    # ----------------------------------------
    # Build and solve
    # ----------------------------------------

    def _add_constraints(self):
        used_weight = sum(
            weight * x_i for weight, x_i in zip(self.instance.weights, self.x)
        )
        self.model.add(used_weight <= self.instance.capacity)

    def _build_model(self):
        self._add_constraints()
        self.set_maximize_value_objective()

    def solve(self) -> KnapsackSolution:
        self.solver.parameters.max_time_in_seconds = self.config.time_limit
        self.solver.parameters.relative_gap_limit = self.config.opt_tol
        self.solver.parameters.log_search_progress = self.config.log_search_progress
        status = self.solver.solve(self.model)
        if status in [cp_model.OPTIMAL, cp_model.FEASIBLE]:
            # If we have a solution, we add it as a hint for the next solve.
            self._set_solution_as_hint()
            return KnapsackSolution(
                selected_items=[
                    i for i in range(self.n) if self.solver.value(self.x[i])
                ],
                objective=self.solver.objective_value,
                upper_bound=self.solver.best_objective_bound,
            )
        return KnapsackSolution(
            selected_items=[], objective=0, upper_bound=float("inf")
        )

In [58]:
def analyze_solution(instance: KnapsackInstance, solution: KnapsackSolution):
    print("Num Selected items:", len(solution.selected_items))
    print(
        "Total weight of selected items:",
        sum(instance.weights[i] for i in solution.selected_items),
    )
    print(
        "Total value of selected items:",
        sum(instance.values[i] for i in solution.selected_items),
    )

In [59]:
import random

instance_size = 800
random_weights = [random.randint(1, 1000) for _ in range(instance_size)]
random_values = [random.randint(1, 1000) for _ in range(instance_size)]
random_capacity = 2 * (sum(random_weights) // 3)

instance = KnapsackInstance(
    weights=random_weights, values=random_values, capacity=random_capacity
)
config = KnapsackSolverConfig(time_limit=15, opt_tol=0.01, log_search_progress=True)
solver = MultiObjectiveKnapsackSolver(instance, config)
solution_1 = solver.solve()

solver.fix_current_objective(
    0.95
)  # maintain at least 95% of the current objective value
solver.set_minimize_weight_objective()  # change the objective to minimize the weight
solution_2 = solver.solve()

print("Solution 1:")
analyze_solution(instance, solution_1)
print("Solution 2:")
analyze_solution(instance, solution_2)


Starting CP-SAT solver v9.10.4067
Parameters: max_time_in_seconds: 15 log_search_progress: true relative_gap_limit: 0.01
Setting number of workers to 16

Initial optimization model '': (model_fingerprint: 0x592646cfd27bed9d)
#Variables: 800 (#bools: 800 in objective)
  - 800 Booleans in [0,1]
#kLinearN: 1 (#terms: 800)

Starting presolve at 0.00s
  2.41e+00s  0.00e+00d  [DetectDominanceRelations] 
  1.06e+00s  0.00e+00d  [DetectDominanceRelations] 
  3.46e+00s  0.00e+00d  [PresolveToFixPoint] #num_loops=3 #num_dual_strengthening=2 
  3.53e-04s  0.00e+00d  [ExtractEncodingFromLinear] 
[Symmetry] Graph for symmetry has 3'135 nodes and 169'080 arcs.
[Symmetry] Symmetry computation done. time: 0.000781933 dtime: 0.00395586
[SAT presolve] num removable Booleans: 0 / 778
[SAT presolve] num trivial clauses: 0
[SAT presolve] [0s] clauses:82984 literals:165968 vars:778 one_side_vars:778 simple_definition:0 singleton_clauses:0
[SAT presolve] [0.0213627s] clauses:82984 literals:165968 vars:778 o

In [60]:
353607 / 372108

0.9502805637073107

In [61]:
for i, v in enumerate(solver.model.proto.variables):
    v_ = solver.model.get_int_var_from_proto_index(i)
    print(v, v_)

name: "x_0"
domain: 0
domain: 1
 x_0
name: "x_1"
domain: 0
domain: 1
 x_1
name: "x_2"
domain: 0
domain: 1
 x_2
name: "x_3"
domain: 0
domain: 1
 x_3
name: "x_4"
domain: 0
domain: 1
 x_4
name: "x_5"
domain: 0
domain: 1
 x_5
name: "x_6"
domain: 0
domain: 1
 x_6
name: "x_7"
domain: 0
domain: 1
 x_7
name: "x_8"
domain: 0
domain: 1
 x_8
name: "x_9"
domain: 0
domain: 1
 x_9
name: "x_10"
domain: 0
domain: 1
 x_10
name: "x_11"
domain: 0
domain: 1
 x_11
name: "x_12"
domain: 0
domain: 1
 x_12
name: "x_13"
domain: 0
domain: 1
 x_13
name: "x_14"
domain: 0
domain: 1
 x_14
name: "x_15"
domain: 0
domain: 1
 x_15
name: "x_16"
domain: 0
domain: 1
 x_16
name: "x_17"
domain: 0
domain: 1
 x_17
name: "x_18"
domain: 0
domain: 1
 x_18
name: "x_19"
domain: 0
domain: 1
 x_19
name: "x_20"
domain: 0
domain: 1
 x_20
name: "x_21"
domain: 0
domain: 1
 x_21
name: "x_22"
domain: 0
domain: 1
 x_22
name: "x_23"
domain: 0
domain: 1
 x_23
name: "x_24"
domain: 0
domain: 1
 x_24
name: "x_25"
domain: 0
domain: 1
 x_25
name: 